In [1]:
# Fama-French 3 factor model replication

# Needed libraries

# The following libraries are skipped for a quick analysis. One can easily make a sql backend to store the data if need be
# The SQL functions were mimicked using Pandas 
# A version utilizing the sql queries is available upon request

#!pip install pandasql
#from sqlite3 import connect
#import pandasql as ps

import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math
import warnings


# The original Fama-French 3 Factor model added 2 factors to CAPM to form the following: 

$ r = R_f + \beta(R_m - R_f) + b_s \cdot SMB + b_v \cdot HML + \alpha $


In [2]:
# To replicate the FF 3 factor model, we must construct the factors from open data sources 

# Dr. William Powley's instructional video advised to directly use the Google collab to SQL backend method 
# but since I have disk space, I'm going to simply convert the csv data to pandas dfs

# Source: https://www.youtube.com/watch?v=mm0DA8zgHIs&t=208s

#ccmfund_df = pd.read_csv('ccmfund1990to2022.csv')

# Start with CRSP data (1990 through 2022)

crsp_df = pd.read_csv("CRSPMonthly1990Through2022.csv")

#ff_3_df = pd.read_csv("FamaFrenchThreeFactorsMonthly.csv")

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3269612979.py:12: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp_df = pd.read_csv("CRSPMonthly1990Through2022.csv")


In [3]:
# Inspect crsp
crsp_df.head()


,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10001,19900131,NaN,11.0,3.0,4920,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.9375,0.125,19900131.0,-0.018519,-0.070114,-0.071947,-0.046408,-0.048037,-0.068817
1,10001,19900228,NaN,11.0,3.0,4920,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900228.0,-0.006289,0.014900,0.010957,0.015434,0.013844,0.008539
2,10001,19900330,NaN,11.0,3.0,4920,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900330.0,0.000000,0.024148,0.021626,0.021315,0.019445,0.024255
3,10001,19900430,NaN,11.0,3.0,4920,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900430.0,0.000000,-0.028283,-0.030562,-0.028116,-0.029637,-0.026887
4,10001,19900531,NaN,11.0,3.0,4920,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,9.7500,NaN,19900531.0,-0.012658,0.088935,0.084648,0.045673,0.043835,0.091989


In [4]:
# This data needs to be cleaned now
# Start with crsp data -- cleaning for relevant variables
column_names_crsp = crsp_df.columns.tolist()
print(column_names_crsp)

['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP', 'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT', 'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT', 'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG', 'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM', 'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD', 'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'ALTPRC', 'SPREAD', 'ALTPRCDT', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn']


In [5]:
# Convert particular relevant columns into the data type they are

crsp_df['PERMNO'] = pd.to_numeric(crsp_df['PERMNO'], errors = 'coerce') #security identifier
crsp_df['date'] = pd.to_datetime(crsp_df['date'], format = "%Y%m%d") #Date identifier
crsp_df['RET'] = pd.to_numeric(crsp_df['RET'], errors = 'coerce') #Returns
crsp_df['SHROUT'] = pd.to_numeric(crsp_df['SHROUT'], errors = 'coerce')#Shares outstanding in thousands
crsp_df['ALTPRC'] = pd.to_numeric(crsp_df['ALTPRC'], errors = 'coerce')#Last traded price in a month
crsp_df['EXCHCD'] = pd.to_numeric(crsp_df['EXCHCD'], errors = 'coerce')#exchange ID
crsp_df['SHRCD'] = pd.to_numeric(crsp_df['SHRCD'], errors = 'coerce')#share code
crsp_df['SICCD'] = pd.to_numeric(crsp_df['SICCD'], errors = 'coerce')#industry code
crsp_df['DLRET'] = pd.to_numeric(crsp_df['DLRET'], errors = 'coerce')#delisting return
crsp_df['DLSTCD'] = pd.to_numeric(crsp_df['DLSTCD'], errors = 'coerce')#delisting code

crsp_df.head()



,PERMNO,date,NAMEENDT,SHRCD,EXCHCD,SICCD,NCUSIP,TICKER,COMNAM,SHRCLS,...,CFACSHR,ALTPRC,SPREAD,ALTPRCDT,RETX,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10001,1990-01-31,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.9375,0.125,19900131.0,-0.018519,-0.070114,-0.071947,-0.046408,-0.048037,-0.068817
1,10001,1990-02-28,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900228.0,-0.006289,0.014900,0.010957,0.015434,0.013844,0.008539
2,10001,1990-03-30,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900330.0,0.000000,0.024148,0.021626,0.021315,0.019445,0.024255
3,10001,1990-04-30,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900430.0,0.000000,-0.028283,-0.030562,-0.028116,-0.029637,-0.026887
4,10001,1990-05-31,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,9.7500,NaN,19900531.0,-0.012658,0.088935,0.084648,0.045673,0.043835,0.091989


In [6]:
# We will get rid of the capitalizations in the column headers just so they can match the CCM fund data

crsp_df.columns = crsp_df.columns.str.lower() #Needed later for merge
crsp_df.head()

,permno,date,nameendt,shrcd,exchcd,siccd,ncusip,ticker,comnam,shrcls,...,cfacshr,altprc,spread,altprcdt,retx,vwretd,vwretx,ewretd,ewretx,sprtrn
0,10001,1990-01-31,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.9375,0.125,19900131.0,-0.018519,-0.070114,-0.071947,-0.046408,-0.048037,-0.068817
1,10001,1990-02-28,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900228.0,-0.006289,0.014900,0.010957,0.015434,0.013844,0.008539
2,10001,1990-03-30,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900330.0,0.000000,0.024148,0.021626,0.021315,0.019445,0.024255
3,10001,1990-04-30,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,-9.8750,0.250,19900430.0,0.000000,-0.028283,-0.030562,-0.028116,-0.029637,-0.026887
4,10001,1990-05-31,NaN,11.0,3.0,4920.0,39040610,GFGC,GREAT FALLS GAS CO,NaN,...,3.0,9.7500,NaN,19900531.0,-0.012658,0.088935,0.084648,0.045673,0.043835,0.091989


In [7]:
# The non SQL route requires some additional adjustments:

# List of columns to apply the SQL Distinct operation on
columns_to_check = ['permno', 'date', 'ret', 'shrout', 'altprc', 'exchcd', 'shrcd', 'siccd', 'dlret', 'dlstcd']

# Drop the columns not in this list 
crsp_df = crsp_df.filter(items=columns_to_check)

# Drop duplicates based on the specified columns
crsp_df = crsp_df.drop_duplicates(subset=columns_to_check)


# Display the updated DataFrame with distinct rows based on the selected columns
print(crsp_df)


         permno       date       ret     shrout      altprc  exchcd  shrcd  \
0         10001 1990-01-31 -0.018519     1022.0    -9.93750     3.0   11.0   
1         10001 1990-02-28 -0.006289     1022.0    -9.87500     3.0   11.0   
2         10001 1990-03-30  0.012658     1027.0    -9.87500     3.0   11.0   
3         10001 1990-04-30  0.000000     1027.0    -9.87500     3.0   11.0   
4         10001 1990-05-31 -0.012658     1027.0     9.75000     3.0   11.0   
...         ...        ...       ...        ...         ...     ...    ...   
2975236   93436 2021-11-30  0.027612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31 -0.076855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -0.113609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28 -0.070768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  0.238009  1033534.0  1077.59998     3.0   11.0   

          siccd  dlret  dlstcd  
0        4920.0    NaN     NaN

In [8]:
# Drop all columns not in columns_to_check
print(crsp_df)

         permno       date       ret     shrout      altprc  exchcd  shrcd  \
0         10001 1990-01-31 -0.018519     1022.0    -9.93750     3.0   11.0   
1         10001 1990-02-28 -0.006289     1022.0    -9.87500     3.0   11.0   
2         10001 1990-03-30  0.012658     1027.0    -9.87500     3.0   11.0   
3         10001 1990-04-30  0.000000     1027.0    -9.87500     3.0   11.0   
4         10001 1990-05-31 -0.012658     1027.0     9.75000     3.0   11.0   
...         ...        ...       ...        ...         ...     ...    ...   
2975236   93436 2021-11-30  0.027612  1004265.0  1144.76001     3.0   11.0   
2975237   93436 2021-12-31 -0.076855  1033534.0  1056.78003     3.0   11.0   
2975238   93436 2022-01-31 -0.113609  1033534.0   936.71997     3.0   11.0   
2975239   93436 2022-02-28 -0.070768  1033534.0   870.42999     3.0   11.0   
2975240   93436 2022-03-31  0.238009  1033534.0  1077.59998     3.0   11.0   

          siccd  dlret  dlstcd  
0        4920.0    NaN     NaN

In [9]:
# Convert returns to percentages

crsp_df['ret'] *= 100
crsp_df['dlret'] *= 100
crsp_df.head()

,permno,date,ret,shrout,altprc,exchcd,shrcd,siccd,dlret,dlstcd
0,10001,1990-01-31,-1.8519,1022.0,-9.9375,3.0,11.0,4920.0,NaN,NaN
1,10001,1990-02-28,-0.6289,1022.0,-9.8750,3.0,11.0,4920.0,NaN,NaN
2,10001,1990-03-30,1.2658,1027.0,-9.8750,3.0,11.0,4920.0,NaN,NaN
3,10001,1990-04-30,0.0000,1027.0,-9.8750,3.0,11.0,4920.0,NaN,NaN
4,10001,1990-05-31,-1.2658,1027.0,9.7500,3.0,11.0,4920.0,NaN,NaN


In [10]:
# Filter columns by date as we only want to keep data after 2018

crsp_df = crsp_df[crsp_df['date'] >= "2018-01-01"]

# Keep only US-based common stocks

crsp_df = crsp_df[(crsp_df['shrcd'] == 10) | (crsp_df['shrcd'] == 11)] # as 10 and 11 are the share codes for US stocks
crsp_df.head()


,permno,date,ret,shrout,altprc,exchcd,shrcd,siccd,dlret,dlstcd
2363,10026,2018-01-31,-8.8191,18678.0,138.44,3.0,11.0,2052.0,NaN,NaN
2364,10026,2018-02-28,-2.9688,18678.0,134.33,3.0,11.0,2052.0,NaN,NaN
2365,10026,2018-03-29,1.9951,18697.0,136.56,3.0,11.0,2052.0,NaN,NaN
2366,10026,2018-04-30,0.6224,18702.0,137.41,3.0,11.0,2052.0,NaN,NaN
2367,10026,2018-05-31,3.0638,18702.0,141.62,3.0,11.0,2052.0,NaN,NaN


In [11]:
# Market cap col

# Calculate market cap in millions

crsp_df["market_cap"] = abs(crsp_df['shrout'] * crsp_df['altprc'])/1000
crsp_df["market_cap"].replace(0,np.NaN)

crsp_df.head()

,permno,date,ret,shrout,altprc,exchcd,shrcd,siccd,dlret,dlstcd,market_cap
2363,10026,2018-01-31,-8.8191,18678.0,138.44,3.0,11.0,2052.0,NaN,NaN,2585.78232
2364,10026,2018-02-28,-2.9688,18678.0,134.33,3.0,11.0,2052.0,NaN,NaN,2509.01574
2365,10026,2018-03-29,1.9951,18697.0,136.56,3.0,11.0,2052.0,NaN,NaN,2553.26232
2366,10026,2018-04-30,0.6224,18702.0,137.41,3.0,11.0,2052.0,NaN,NaN,2569.84182
2367,10026,2018-05-31,3.0638,18702.0,141.62,3.0,11.0,2052.0,NaN,NaN,2648.57724


In [12]:
# We need a way to identify the exchanges between NYSE, AMEX, NASDAQ while ignoring stocks which dont trade in the 3

crsp_df['exchange'] = ""

for i in crsp_df.index:
    
    if crsp_df["exchcd"][i] in [1,31]:
        crsp_df["exchange"][i] = "NYSE"
    
    elif crsp_df["exchcd"][i] in [2,32]:
        crsp_df["exchange"][i] = "AMEX"

    elif crsp_df["exchcd"][i] in [3,33]:
        crsp_df["exchange"][i] = "NASDAQ"
    
    else:
        crsp_df['exchange'][i] = "Other"
        
crsp_df = crsp_df[crsp_df['exchange'] != "Other"] #Ignore others

crsp_df.head()

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/2441149839.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_df["exchange"][i] = "NASDAQ"
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/2441149839.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_df["exchange"][i] = "AMEX"
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/2441149839.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

,permno,date,ret,shrout,altprc,exchcd,shrcd,siccd,dlret,dlstcd,market_cap,exchange
2363,10026,2018-01-31,-8.8191,18678.0,138.44,3.0,11.0,2052.0,NaN,NaN,2585.78232,NASDAQ
2364,10026,2018-02-28,-2.9688,18678.0,134.33,3.0,11.0,2052.0,NaN,NaN,2509.01574,NASDAQ
2365,10026,2018-03-29,1.9951,18697.0,136.56,3.0,11.0,2052.0,NaN,NaN,2553.26232,NASDAQ
2366,10026,2018-04-30,0.6224,18702.0,137.41,3.0,11.0,2052.0,NaN,NaN,2569.84182,NASDAQ
2367,10026,2018-05-31,3.0638,18702.0,141.62,3.0,11.0,2052.0,NaN,NaN,2648.57724,NASDAQ


In [27]:
# Add adjusted returns column

crsp_df['ret_adj'] = ""

for i in crsp_df.index:
    
    if pd.isnull(crsp_df['dlstcd'][i]):
        crsp_df['ret_adj'][i] = crsp_df['ret'][i] # If delisting code is null, then adjusted ret is ret
    
    elif(pd.notnull(crsp_df['dlstcd'][i])) and (pd.notnull(crsp_df['dlret'][i])):
        crsp_df['ret_adj'][i] = crsp_df['dlret'][i] #If not null, then adj ret is delisting return
        
    elif(551 <= crsp_df['dlstcd'][i] <= 574) or (crsp_df['dlstcd'][i] in [500,520,580,584]):
        crsp_df["ret_adj"][i] = -30 # specific delisting codes end up with negative numeric
    
    else:
        crsp_df['ret_adj'][i] = -100
    

crsp_df.head()




/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/1650528840.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_df['ret_adj'][i] = crsp_df['ret'][i] # If delisting code is null, then adjusted ret is ret
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/1650528840.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crsp_df['ret_adj'][i] = -100
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/1650528840.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

,permno,date,ret,shrout,altprc,exchcd,shrcd,siccd,dlret,dlstcd,market_cap,exchange,ret_adj
2363,10026,2018-01-31,-8.8191,18678.0,138.44,3.0,11.0,2052.0,NaN,NaN,2585.78232,NASDAQ,-8.8191
2364,10026,2018-02-28,-2.9688,18678.0,134.33,3.0,11.0,2052.0,NaN,NaN,2509.01574,NASDAQ,-2.9688
2365,10026,2018-03-29,1.9951,18697.0,136.56,3.0,11.0,2052.0,NaN,NaN,2553.26232,NASDAQ,1.9951
2366,10026,2018-04-30,0.6224,18702.0,137.41,3.0,11.0,2052.0,NaN,NaN,2569.84182,NASDAQ,0.6224
2367,10026,2018-05-31,3.0638,18702.0,141.62,3.0,11.0,2052.0,NaN,NaN,2648.57724,NASDAQ,3.0638


In [28]:
# We need 10 columns for crsp_df_Final: PERMNO date RET SHROUT ALTPRC EXCHCD SICCD market_cap exchange ret_adj

# List of columns to keep
columns_to_keep = ['permno', 'date', 'ret', 'shrout', 'altprc', 'exchcd', 'siccd', 'market_cap', 'exchange', 'ret_adj']

# Filter the DataFrame to only keep the specified columns
crsp_df_Final = crsp_df.filter(items=columns_to_keep)

# Display the updated DataFrame
print(crsp_df_Final)

# The df is correct at 193375 x 10

         permno       date      ret     shrout      altprc  exchcd   siccd  \
2363      10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
2364      10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2365      10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
2366      10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
2367      10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...         ...        ...      ...        ...         ...     ...     ...   
2975236   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
2975237   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
2975238   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
2975239   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
2975240   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

           market_cap exchange  ret_adj  
2363     2.585782e+03

In [29]:
# Reset the index 

crsp_df_Final = crsp_df_Final.reset_index(drop=True)
print(crsp_df_Final)


        permno       date      ret     shrout      altprc  exchcd   siccd  \
0        10026 2018-01-31  -8.8191    18678.0   138.44000     3.0  2052.0   
1        10026 2018-02-28  -2.9688    18678.0   134.33000     3.0  2052.0   
2        10026 2018-03-29   1.9951    18697.0   136.56000     3.0  2052.0   
3        10026 2018-04-30   0.6224    18702.0   137.41000     3.0  2052.0   
4        10026 2018-05-31   3.0638    18702.0   141.62000     3.0  2052.0   
...        ...        ...      ...        ...         ...     ...     ...   
193370   93436 2021-11-30   2.7612  1004265.0  1144.76001     3.0  9999.0   
193371   93436 2021-12-31  -7.6855  1033534.0  1056.78003     3.0  9999.0   
193372   93436 2022-01-31 -11.3609  1033534.0   936.71997     3.0  9999.0   
193373   93436 2022-02-28  -7.0768  1033534.0   870.42999     3.0  9999.0   
193374   93436 2022-03-31  23.8009  1033534.0  1077.59998     3.0  9999.0   

          market_cap exchange  ret_adj  
0       2.585782e+03   NASDAQ  -8.

In [39]:
# Moving on to the compustat merged (CCM) data set

ccm_df = pd.read_csv('ccmfund1990to2022.csv')

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/1485621986.py:3: DtypeWarning: Columns (2,17,19,24,33,37,952,953,955,956,957,962,976,977,982) have mixed types. Specify dtype option on import or set low_memory=False.
  ccm_df = pd.read_csv('ccmfund1990to2022.csv')


In [40]:
column_names_ccm = ccm_df.columns.tolist()
print(column_names_ccm)

['GVKEY', 'LINKPRIM', 'LIID', 'LINKTYPE', 'LPERMNO', 'LPERMCO', 'LINKDT', 'LINKENDDT', 'datadate', 'fyear', 'indfmt', 'consol', 'popsrc', 'datafmt', 'tic', 'cusip', 'conm', 'acctchg', 'acctstd', 'acqmeth', 'adrr', 'ajex', 'ajp', 'bspr', 'compst', 'curcd', 'curncd', 'currtr', 'curuscn', 'final', 'fyr', 'ismod', 'ltcm', 'ogm', 'pddur', 'scf', 'src', 'stalt', 'udpl', 'upd', 'apdedate', 'fdate', 'pdate', 'acchg', 'acco', 'accrt', 'acdo', 'aco', 'acodo', 'acominc', 'acox', 'acoxar', 'acqao', 'acqcshi', 'acqgdwl', 'acqic', 'acqintan', 'acqinvt', 'acqlntal', 'acqniintc', 'acqppe', 'acqsc', 'act', 'adpac', 'aedi', 'afudcc', 'afudci', 'aldo', 'am', 'amc', 'amdc', 'amgw', 'ano', 'ao', 'aocidergl', 'aociother', 'aocipen', 'aocisecgl', 'aodo', 'aol2', 'aoloch', 'aox', 'ap', 'apalch', 'apb', 'apc', 'apofs', 'aqa', 'aqc', 'aqd', 'aqeps', 'aqi', 'aqp', 'aqpl1', 'aqs', 'arb', 'arc', 'arce', 'arced', 'arceeps', 'artfs', 'at', 'aul3', 'autxr', 'balr', 'banlr', 'bast', 'bastr', 'batr', 'bcef', 'bclr', 'b

In [41]:
# Fetch only the distinct values from relevant columns

# List of columns to apply the SQL Distinct operation on
columns_to_check_ccm = ['GVKEY', 'LPERMNO', 'datadate', 'LINKTYPE', 
                        'LINKENDDT', 'seq', 'ceq', 'at', 'lt', 'txditc',
                       'txdb','itcb', 'pstkrv', 'pstk','pstkl', 'indfmt', 'datafmt']

# Drop the columns not in this list 
ccm_df = ccm_df.filter(items=columns_to_check_ccm)

# Drop duplicates based on the specified columns
ccm_df = ccm_df.drop_duplicates(subset=columns_to_check_ccm)


# Display the updated DataFrame with distinct rows based on the selected columns
print(ccm_df)

         GVKEY  LPERMNO  datadate LINKTYPE LINKENDDT       seq       ceq  \
0         1004    54594  19910531       LU         E   193.778   193.778   
1         1004    54594  19920531       LU         E   196.737   196.737   
2         1004    54594  19930531       LU         E   189.216   189.216   
3         1004    54594  19940531       LU         E   189.488   189.488   
4         1004    54594  19950531       LU         E   197.119   197.119   
...        ...      ...       ...      ...       ...       ...       ...   
209121  345980    20333  20201231       LC         E  1027.000  1027.000   
209122  345980    20333  20211231       LC         E   818.000   818.000   
209123  347007    15533  20211231       LC         E  -242.173  -242.173   
209124  347085    21069  20220228       LC         E   139.431   139.431   
209125  349530    17324  20220228       LC         E    64.231    64.231   

              at        lt  txditc    txdb  itcb  pstkrv  pstk  pstkl indfmt  \
0      

In [42]:
#Convert variables

ccm_df['GVKEY'] = pd.to_numeric(ccm_df['GVKEY'], errors = 'coerce') #firm identifier
ccm_df['LPERMNO'] = pd.to_numeric(ccm_df['LPERMNO'], errors = 'coerce') #stock identifier
ccm_df['datadate'] = pd.to_datetime(ccm_df['datadate'], format = '%Y%m%d')# date of report
ccm_df['LINKTYPE'] = ccm_df['LINKTYPE'].apply(str) #link type
ccm_df['LINKENDDT'] = pd.to_datetime(ccm_df['LINKENDDT'], format = '%Y%m%d', errors = 'coerce') #link's date of expiration
ccm_df['seq'] = pd.to_numeric(ccm_df['seq'], errors = 'coerce')#stockholder equit
ccm_df['ceq'] = pd.to_numeric(ccm_df['ceq'], errors = 'coerce')# total equity
ccm_df['at'] = pd.to_numeric(ccm_df['at'], errors = 'coerce')#total assets
ccm_df['lt'] = pd.to_numeric(ccm_df['lt'], errors = 'coerce')#total liabilities
ccm_df['txditc'] = pd.to_numeric(ccm_df['txditc'], errors = 'coerce')#deferred taxes and investment tax credit
ccm_df['txdb'] = pd.to_numeric(ccm_df['txdb'], errors = 'coerce')#deferred taxes
ccm_df['itcb'] = pd.to_numeric(ccm_df['itcb'], errors = 'coerce')#investment tax credit
ccm_df['pstkrv'] = pd.to_numeric(ccm_df['pstkrv'], errors = 'coerce')#preferred stock redemption val
ccm_df['pstk'] = pd.to_numeric(ccm_df['pstk'], errors = 'coerce')#preferred stock liquidating val
ccm_df['pstkl'] = pd.to_numeric(ccm_df['pstkl'], errors = 'coerce')#preffered stock par val

ccm_df['indfmt'] = ccm_df['indfmt'].apply(str) #industry format
ccm_df['datafmt'] = ccm_df['datafmt'].apply(str) #data format


In [43]:
# filter the columns 
ccm_df = ccm_df[ccm_df['datadate'] >= "2018-01-01 00:00:00"] 

#keep correct formats

ccm_df = ccm_df[ccm_df['indfmt'] == "INDL"]
ccm_df = ccm_df[ccm_df['datafmt'] == "STD"]

#Only keeo valid links

ccm_df = ccm_df[(ccm_df['LINKTYPE'] == 'LU') | (ccm_df['LINKTYPE'] == 'LC')]

# only keep links active at datadate

ccm_df = ccm_df[(ccm_df['datadate'] <= ccm_df['LINKENDDT']) | pd.isnull(ccm_df['LINKENDDT'])]


In [47]:
# Add book value (non SQL method)

# Rename columns as required
ccm_df = ccm_df.rename(columns={'GVKEY': 'gvkey', 'LPERMNO': 'permno'})

# Create 'book_value' column using coalesce logic
ccm_df['book_value'] = (
    ccm_df['seq'].fillna(ccm_df['ceq'] + ccm_df['pstk']).fillna(ccm_df['at'] - ccm_df['lt']) +
    ccm_df['txditc'].fillna(ccm_df['txdb'] + ccm_df['itcb']).fillna(0) -
    ccm_df['pstkrv'].fillna(ccm_df['pstkl']).fillna(ccm_df['pstk']).fillna(0)
)

# Select distinct rows based on 'gvkey', 'permno', and 'datadate', but keep all other columns
distinct_df = ccm_df.drop_duplicates(subset=['gvkey', 'permno', 'datadate'])
print(distinct_df)

        gvkey  permno   datadate LINKTYPE LINKENDDT        seq        ceq  \
0       12825   10026 2018-09-30       LU       NaT    759.091    759.091   
1       12825   10026 2019-09-30       LU       NaT    833.751    833.751   
2       12825   10026 2020-09-30       LU       NaT    809.498    809.498   
3       12825   10026 2021-09-30       LU       NaT    845.654    845.654   
4       12096   10028 2018-12-31       LC       NaT      8.413      8.413   
...       ...     ...        ...      ...       ...        ...        ...   
22288  184259   93434 2021-06-30       LC       NaT     74.424     74.424   
22289  184996   93436 2018-12-31       LC       NaT   4923.243   4923.243   
22290  184996   93436 2019-12-31       LC       NaT   6618.000   6618.000   
22291  184996   93436 2020-12-31       LC       NaT  22225.000  22225.000   
22292  184996   93436 2021-12-31       LC       NaT  30189.000  30189.000   

              at         lt   txditc     txdb  itcb  pstkrv  pstk  pstkl  \

In [48]:
distinct_df['book_value'] = pd.to_numeric(distinct_df['book_value'], errors = 'coerce')
print(distinct_df)

        gvkey  permno   datadate LINKTYPE LINKENDDT        seq        ceq  \
0       12825   10026 2018-09-30       LU       NaT    759.091    759.091   
1       12825   10026 2019-09-30       LU       NaT    833.751    833.751   
2       12825   10026 2020-09-30       LU       NaT    809.498    809.498   
3       12825   10026 2021-09-30       LU       NaT    845.654    845.654   
4       12096   10028 2018-12-31       LC       NaT      8.413      8.413   
...       ...     ...        ...      ...       ...        ...        ...   
22288  184259   93434 2021-06-30       LC       NaT     74.424     74.424   
22289  184996   93436 2018-12-31       LC       NaT   4923.243   4923.243   
22290  184996   93436 2019-12-31       LC       NaT   6618.000   6618.000   
22291  184996   93436 2020-12-31       LC       NaT  22225.000  22225.000   
22292  184996   93436 2021-12-31       LC       NaT  30189.000  30189.000   

              at         lt   txditc     txdb  itcb  pstkrv  pstk  pstkl  \

In [49]:
# Need to keep last observation for each year

distinct_df["datadate"] = pd.to_datetime(distinct_df["datadate"])
distinct_df['year'] = distinct_df['datadate'].dt.year
distinct_df = distinct_df.sort_values(by = ['permno', 'datadate'], ascending = True)
distinct_df = ccm_df.reset_index()

to_be_dropped = []

for i in range(len(ccm_df) - 1):
    
    if(distinct_df['permno'][i] == distinct_df["permno"][i+1]) and (distinct_df['year'][i] == distinct_df['year'][i+1]):
        to_be_dropped.append(i)

distinct_df.drop(to_be_dropped, inplace = True, axis = 0)

if 'index' in distinct_df.columns:
    distinct_df.drop('index', inplace=True, axis=1)



In [50]:
# Need to drop missing vals

distinct_df.replace([np.inf, -np.inf], np.nan, inplace = True)
distinct_df = distinct_df.dropna()

# Reindex 

distinct_df = distinct_df.reset_index()
distinct_df.drop('index', inplace = True, axis = 1)



In [51]:
distinct_df['year'] = pd.to_numeric(distinct_df['year'], errors = "coerce")
distinct_df['reference_date'] = distinct_df['year'] +1 
distinct_df['reference_date'] = distinct_df['reference_date'].apply(str)
distinct_df['reference_date'] += "-06-01"
distinct_df['reference_date'] = pd.to_datetime(distinct_df['reference_date'])
distinct_df.drop('year', inplace = True, axis = 1)



In [52]:
print(distinct_df.columns)

Index(['gvkey', 'permno', 'datadate', 'LINKTYPE', 'LINKENDDT', 'seq', 'ceq',
       'at', 'lt', 'txditc', 'txdb', 'itcb', 'pstkrv', 'pstk', 'pstkl',
       'indfmt', 'datafmt', 'book_value', 'reference_date'],
      dtype='object')


In [30]:
# Next we make the stocks df 

# Select distinct permno, date, exchange, ret_adj as ret, market_cap from crsp_df_Final

stocks_df = crsp_df_Final[['permno', 'date', 'exchange', 'ret_adj', 'market_cap']].drop_duplicates()
stocks_df.rename(columns={'ret_adj': 'ret'}, inplace=True)
# Drop missing/infinite values

stocks_df.replace([np.inf, -np.inf], np.nan, inplace = True)

stocks_df = stocks_df.dropna()

print(stocks_df)

        permno       date exchange       ret    market_cap
0        10026 2018-01-31   NASDAQ   -8.8191  2.585782e+03
1        10026 2018-02-28   NASDAQ   -2.9688  2.509016e+03
2        10026 2018-03-29   NASDAQ    1.9951  2.553262e+03
3        10026 2018-04-30   NASDAQ    0.6224  2.569842e+03
4        10026 2018-05-31   NASDAQ    3.0638  2.648577e+03
...        ...        ...      ...       ...           ...
193370   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06
193371   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06
193372   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05
193373   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05
193374   93436 2022-03-31   NASDAQ -100.0000  1.113736e+06

[191617 rows x 5 columns]


In [31]:
# Add Reference Date Column

# Last 6/1
stocks_df['date'] = pd.to_datetime(stocks_df['date'])
stocks_df['reference_date'] = ""

for i in stocks_df.index:
    
    if stocks_df['date'][i].month < 6:
        
        stocks_df['reference_date'][i] = str(stocks_df['date'][i].year - 1) + "-06-01"
    
    else:
        
        stocks_df['reference_date'][i] = str(stocks_df['date'][i].year + 1) + "-06-01"
        
stocks_df['reference_date'] = pd.to_datetime(stocks_df['reference_date'])

print(stocks_df)

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/1918167356.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_df['reference_date'][i] = str(stocks_df['date'][i].year - 1) + "-06-01"
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/1918167356.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks_df['reference_date'][i] = str(stocks_df['date'][i].year + 1) + "-06-01"


        permno       date exchange       ret    market_cap reference_date
0        10026 2018-01-31   NASDAQ   -8.8191  2.585782e+03     2017-06-01
1        10026 2018-02-28   NASDAQ   -2.9688  2.509016e+03     2017-06-01
2        10026 2018-03-29   NASDAQ    1.9951  2.553262e+03     2017-06-01
3        10026 2018-04-30   NASDAQ    0.6224  2.569842e+03     2017-06-01
4        10026 2018-05-31   NASDAQ    3.0638  2.648577e+03     2017-06-01
...        ...        ...      ...       ...           ...            ...
193370   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06     2022-06-01
193371   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06     2022-06-01
193372   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05     2021-06-01
193373   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05     2021-06-01
193374   93436 2022-03-31   NASDAQ -100.0000  1.113736e+06     2021-06-01

[191617 rows x 6 columns]


In [53]:
# Merge compustate data into stocks data

merged_df = pd.merge(
    stocks_df,
    distinct_df[['permno', 'gvkey', 'datadate', 'reference_date', 'book_value']],  # Selecting only the required columns from ccm_df
    on=['permno', 'reference_date'],  # Merging on 'permno' and 'reference_date'
    how='left'  # Left join
)



stocks_df_merged = merged_df.drop_duplicates()
if 'index' in stocks_df_merged.columns:
    stocks_df_merged.drop('index', inplace=True, axis=1)

print(stocks_df_merged)

        permno       date exchange       ret    market_cap reference_date  \
0        10026 2018-01-31   NASDAQ   -8.8191  2.585782e+03     2017-06-01   
1        10026 2018-02-28   NASDAQ   -2.9688  2.509016e+03     2017-06-01   
2        10026 2018-03-29   NASDAQ    1.9951  2.553262e+03     2017-06-01   
3        10026 2018-04-30   NASDAQ    0.6224  2.569842e+03     2017-06-01   
4        10026 2018-05-31   NASDAQ    3.0638  2.648577e+03     2017-06-01   
...        ...        ...      ...       ...           ...            ...   
191612   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06     2022-06-01   
191613   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06     2022-06-01   
191614   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05     2021-06-01   
191615   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05     2021-06-01   
191616   93436 2022-03-31   NASDAQ -100.0000  1.113736e+06     2021-06-01   

        gvkey datadate  book_value  
0         NaN      NaT         NaN  
1

In [54]:
stocks_df_merged['date'] = pd.to_datetime(stocks_df_merged['date'])

# Filter for December dates
stocksMarketEquity = stocks_df_merged[stocks_df_merged['date'].dt.month == 12]

# Create 'reference_date' for next year
stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].astype(str) 
stocksMarketEquity['reference_date'] += "-06-01"

stocksMarketEquity['reference_date'] = pd.to_datetime(stocksMarketEquity['reference_date'])

# Select relevant columns and drop duplicates
stocksMarketEquity_f = stocksMarketEquity[['permno', 'reference_date', 'market_cap']]
stocksMarketEquity_f.rename(columns={'market_cap': 'market_equity'}, inplace=True)

# Merge stocks_df_merged with stocksMarketEquity_f
stocks2018to2022 = pd.merge(
    stocks_df_merged,
    stocksMarketEquity_f[['permno', 'reference_date','market_equity']],  # Use stocksMarketEquity_f directly
    on=['permno', 'reference_date'],  # Merging on these keys
    how='left'  # Left join
).drop_duplicates()

if 'index' in stocks2018to2022.columns:
    stocks2018to2022.drop('index', inplace=True, axis=1)

# Drop rows with NaN values
stocks2018to2022.dropna()

# Reset the index if needed
stocks2018to2022.reset_index()

if 'index' in stocks2018to2022.columns:
    stocks2018to2022.drop('index', inplace=True, axis=1)
print(stocks2018to2022)

        permno       date exchange       ret    market_cap reference_date  \
0        10026 2018-01-31   NASDAQ   -8.8191  2.585782e+03     2017-06-01   
1        10026 2018-02-28   NASDAQ   -2.9688  2.509016e+03     2017-06-01   
2        10026 2018-03-29   NASDAQ    1.9951  2.553262e+03     2017-06-01   
3        10026 2018-04-30   NASDAQ    0.6224  2.569842e+03     2017-06-01   
4        10026 2018-05-31   NASDAQ    3.0638  2.648577e+03     2017-06-01   
...        ...        ...      ...       ...           ...            ...   
191612   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06     2022-06-01   
191613   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06     2022-06-01   
191614   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05     2021-06-01   
191615   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05     2021-06-01   
191616   93436 2022-03-31   NASDAQ -100.0000  1.113736e+06     2021-06-01   

        gvkey datadate  book_value  market_equity  
0         NaN      NaT 

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/2427489180.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksMarketEquity['reference_date'] = stocksMarketEquity['date'].dt.year + 1
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/2427489180.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocksMarketEquity['reference_date'] = stocksMarketEquity['reference_date'].astype(str)
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/2427489180.py:9: SettingWith

In [55]:
# Add book to market ratio and market cap weirght

stocks2018to2022['bm_ratio'] = stocks2018to2022['book_value']/stocks2018to2022['market_equity']

stocks2018to2022['date'] = pd.to_datetime(stocks2018to2022['date'])

stocksWeight = stocks2018to2022[stocks2018to2022['date'].dt.month == 6]

# Merge stocks2018to2022 with stockweights

stocks2018to2022_f = pd.merge(
    stocks2018to2022,
    stocksWeight[['permno', 'reference_date', 'market_cap']],  
    on=['permno', 'reference_date'],  # Merging on these keys
    how='left',  # Left join
    suffixes=('', '_weight')
)


if 'index' in stocks2018to2022_f.columns:
    stocks2018to2022_f.drop('index', inplace=True, axis=1)
    
print(stocks2018to2022_f)

        permno       date exchange       ret    market_cap reference_date  \
0        10026 2018-01-31   NASDAQ   -8.8191  2.585782e+03     2017-06-01   
1        10026 2018-02-28   NASDAQ   -2.9688  2.509016e+03     2017-06-01   
2        10026 2018-03-29   NASDAQ    1.9951  2.553262e+03     2017-06-01   
3        10026 2018-04-30   NASDAQ    0.6224  2.569842e+03     2017-06-01   
4        10026 2018-05-31   NASDAQ    3.0638  2.648577e+03     2017-06-01   
...        ...        ...      ...       ...           ...            ...   
191612   93436 2021-11-30   NASDAQ    2.7612  1.149642e+06     2022-06-01   
191613   93436 2021-12-31   NASDAQ   -7.6855  1.092218e+06     2022-06-01   
191614   93436 2022-01-31   NASDAQ  -11.3609  9.681319e+05     2021-06-01   
191615   93436 2022-02-28   NASDAQ   -7.0768  8.996190e+05     2021-06-01   
191616   93436 2022-03-31   NASDAQ -100.0000  1.113736e+06     2021-06-01   

        gvkey datadate  book_value  market_equity  bm_ratio  market_cap_wei

In [56]:
# Filter data

stocks2018to2022_f.replace([np.inf, -np.inf], np.nan, inplace = True)

stocks2018to2022_f = stocks2018to2022_f.dropna()

stocks2018to2022_f['date'] = pd.to_datetime(stocks2018to2022_f['date'])
stocks2018to2022_f = stocks2018to2022_f[stocks2018to2022_f['date'] >= "2018-01-01"]

if 'index' in stocks2018to2022_f.columns:
    stocks2018to2022_f.drop('index', inplace=True, axis=1)

print(stocks2018to2022_f)

# This is off but keep going (lots of missing rows could be fault of join/NAs)

        permno       date exchange      ret  market_cap reference_date  \
518      10180 2018-06-29   NASDAQ  18.7545  2080.45236     2019-06-01   
519      10180 2018-07-31   NASDAQ  11.6335  2323.35252     2019-06-01   
520      10180 2018-08-31   NASDAQ -15.2808  1968.32619     2019-06-01   
521      10180 2018-09-28   NASDAQ -17.2721  1628.88616     2019-06-01   
522      10180 2018-10-31   NASDAQ -48.6132   837.03164     2019-06-01   
...        ...        ...      ...      ...         ...            ...   
191398   93422 2020-11-30     NYSE  78.8889   389.97581     2021-06-01   
191399   93422 2020-12-31     NYSE  48.4472   580.61704     2021-06-01   
191400   93422 2021-01-29     NYSE  19.2469   692.36760     2020-06-01   
191401   93422 2021-02-26     NYSE  20.7018   834.42704     2020-06-01   
191402   93422 2021-03-31     NYSE  18.3285   989.66928     2020-06-01   

           gvkey   datadate  book_value  market_equity  bm_ratio  \
518      14304.0 2018-12-31     444.432    

In [57]:
# Size Sort
stocks2018to2022_f['date'] = pd.to_datetime(stocks2018to2022_f['date'])
sizebreakpoints = stocks2018to2022_f[(stocks2018to2022_f['date'].dt.month == 6) & (stocks2018to2022_f['exchange'] == 'NYSE')]
sizebreakpoints = sizebreakpoints[['permno', 'reference_date', 'market_cap']].drop_duplicates()

sizebreakpoints['reference_date'] = pd.to_datetime(sizebreakpoints['reference_date'])

g = sizebreakpoints.groupby(['reference_date'])

print(sizebreakpoints)



        permno reference_date  market_cap
5360     11707     2019-06-01  1723.75570
7404     12340     2019-06-01   654.92100
7841     12374     2019-06-01  1425.48093
7853     12374     2020-06-01  2267.22540
7865     12374     2021-06-01  5665.49750
...        ...            ...         ...
191335   93420     2019-06-01  4115.48476
191347   93420     2020-06-01  1829.14176
191369   93422     2019-06-01  2914.30008
191381   93422     2020-06-01  1721.06535
191393   93422     2021-06-01   312.41478

[288 rows x 3 columns]


In [58]:
# Create disze sorts data and add median size col

sizesorts =stocks2018to2022_f[(stocks2018to2022_f['date'].dt.month == 6)]

# Calculate median size

sizesorts['size_median'] = ""

for i in sizesorts.index:
    sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
                                              
warnings.filterwarnings("ignore")
                                        
print(sizesorts)

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'] = ""
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:10: SettingWithCopyWarning: 
A value is trying to be set

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/374878

        permno       date exchange      ret  market_cap reference_date  \
518      10180 2018-06-29   NASDAQ  18.7545  2080.45236     2019-06-01   
530      10180 2019-06-28   NASDAQ  25.0000   649.45620     2020-06-01   
730      10239 2018-06-29   NASDAQ   4.7210   302.07200     2019-06-01   
742      10239 2019-06-28   NASDAQ   1.2828   207.25884     2020-06-01   
754      10239 2020-06-30   NASDAQ   9.0449   174.96270     2021-06-01   
...        ...        ...      ...      ...         ...            ...   
191335   93420 2018-06-29     NYSE  -0.4605  4115.48476     2019-06-01   
191347   93420 2019-06-28     NYSE   9.2308  1829.14176     2020-06-01   
191369   93422 2018-06-29     NYSE   1.4061  2914.30008     2019-06-01   
191381   93422 2019-06-28     NYSE   4.6310  1721.06535     2020-06-01   
191393   93422 2020-06-30     NYSE  51.1246   312.41478     2021-06-01   

           gvkey   datadate  book_value  market_equity  bm_ratio  \
518      14304.0 2018-12-31     444.432    

/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/3748780366.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sizesorts['size_median'][i] = g.get_group(g.get_group(sizesorts['reference_date'][i]).reset_index()['reference_date'][0])['market_cap'].quantile(0.5)
/var/folders/bq/c6jbp0zd15s5tvy17rx3_3_w0000gn/T/ipykernel_3877/374878

In [59]:
# Sort firms based on size 

sizesorts['size_portfolio'] = ""

for i in sizesorts.index:
    if sizesorts['market_cap'][i] > sizesorts['size_median'][i]:
        sizesorts['size_portfolio'][i] = "B"
    else:
        sizesorts['size_portfolio'][i] = "S"


# Merge into stocks df

stocks2018to2022_f =pd.merge(
stocks2018to2022_f,
sizesorts[['permno', 'reference_date','size_portfolio']],
 on=['permno', 'reference_date'],  # Merging on these keys
    how='left'
    )

print(stocks2018to2022_f)

      permno       date exchange      ret  market_cap reference_date  \
0      10180 2018-06-29   NASDAQ  18.7545  2080.45236     2019-06-01   
1      10180 2018-07-31   NASDAQ  11.6335  2323.35252     2019-06-01   
2      10180 2018-08-31   NASDAQ -15.2808  1968.32619     2019-06-01   
3      10180 2018-09-28   NASDAQ -17.2721  1628.88616     2019-06-01   
4      10180 2018-10-31   NASDAQ -48.6132   837.03164     2019-06-01   
...      ...        ...      ...      ...         ...            ...   
8617   93422 2020-11-30     NYSE  78.8889   389.97581     2021-06-01   
8618   93422 2020-12-31     NYSE  48.4472   580.61704     2021-06-01   
8619   93422 2021-01-29     NYSE  19.2469   692.36760     2020-06-01   
8620   93422 2021-02-26     NYSE  20.7018   834.42704     2020-06-01   
8621   93422 2021-03-31     NYSE  18.3285   989.66928     2020-06-01   

         gvkey   datadate  book_value  market_equity  bm_ratio  \
0      14304.0 2018-12-31     444.432      425.41788  1.044695   
1  